In [18]:
import pandas as pd
import numpy as np
import json
import math

from sklearn import tree
import sklearn
import graphviz

In [107]:
loans = pd.read_csv('lending-club-data.csv', low_memory = False)

In [110]:
loans.head(2)

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,...,sub_grade_num,delinq_2yrs_zero,pub_rec_zero,collections_12_mths_zero,short_emp,payment_inc_ratio,final_d,last_delinq_none,last_record_none,last_major_derog_none
0,1077501,1296599,5000,5000,4975,36 months,10.65,162.87,B,B2,...,0.4,1.0,1.0,1.0,0,8.1435,20141201T000000,1,1,1
1,1077430,1314167,2500,2500,2500,60 months,15.27,59.83,C,C4,...,0.8,1.0,1.0,1.0,1,2.3932,20161201T000000,1,1,1


In [66]:
di = {1: -1, 0: 1}

In [111]:
loans['safe_loans'] = loans['bad_loans'].map(di)

In [112]:
loans[['safe_loans', 'bad_loans']].head()

,safe_loans,bad_loans
0,1,0
1,-1,1
2,1,0
3,1,0
4,1,0


In [113]:
loans['safe_loans'].value_counts()/len(loans['safe_loans'])

 1    0.811185
-1    0.188815
Name: safe_loans, dtype: float64

In [114]:
features = ['grade',                     # grade of the loan
            'sub_grade',                 # sub-grade of the loan
            'short_emp',                 # one year or less of employment
            'emp_length_num',            # number of years of employment
            'home_ownership',            # home_ownership status: own, mortgage or rent
            'dti',                       # debt to income ratio
            'purpose',                   # the purpose of the loan
            'term',                      # the term of the loan
            'last_delinq_none',          # has borrower had a delinquincy
            'last_major_derog_none',     # has borrower had 90 day or worse rating
            'revol_util',                # percent of available credit being used
            'total_rec_late_fee',        # total late fees received to day
           ]

target = 'safe_loans'                    # prediction target (y) (+1 means safe, -1 is risky)

# Extract the feature columns and target column
loans = loans[features + [target]]
loans1 = loans

In [115]:
len(loans.iloc[0,:])

13

In [130]:
loans1.dtypes

grade                     object
sub_grade                 object
short_emp                  int64
emp_length_num             int64
home_ownership            object
dti                      float64
purpose                   object
term                      object
last_delinq_none           int64
last_major_derog_none      int64
revol_util               float64
total_rec_late_fee       float64
safe_loans                 int64
dtype: object

In [134]:
features = loans.columns.values

categorical_variables = []
for feat_name, feat_type in zip(dict(loans1.dtypes).keys(), dict(
    loans1.dtypes).values()) :
    if feat_type == object:
        categorical_variables.append(feat_name)

for feature in categorical_variables:
    loans_data_one_hot_encoded = pd.get_dummies(loans1[feature],
                                                prefix=feature)
    loans1 =pd.concat([loans1,loans_data_one_hot_encoded],axis=1)
    del loans1[feature] 
   

In [135]:
loans1 = loans1.fillna(0)

In [139]:
with open ('module-5-assignment-1-train-idx.json') as json_data:
    train_index = json.load(json_data)
    
with open ('module-5-assignment-1-validation-idx.json') as json_data:
    test_index = json.load(json_data)
    
    
train_data = loans1.iloc[train_index]
validation_data = loans1.iloc[test_index]

train_data1 = train_data
validation_data1 = validation_data

DecisionTreeClassifier(criterion=’gini’, splitter=’best’, max_depth=None, min_samples_split=2, min_samples_leaf=1, min_weight_fraction_leaf=0.0, max_features=None, random_state=None, max_leaf_nodes=None, min_impurity_decrease=0.0, min_impurity_split=None, class_weight=None, presort=False)

In [140]:
decision_tree_model = tree.DecisionTreeClassifier(max_depth = 6)

In [141]:
decision_tree_model.fit(train_data.drop('safe_loans', axis = 1), train_data['safe_loans'])

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=6,
            max_features=None, max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            presort=False, random_state=None, splitter='best')

In [142]:
small_model = tree.DecisionTreeClassifier(max_depth = 2)
small_model.fit(train_data.drop('safe_loans', axis = 1), train_data['safe_loans'])


DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=2,
            max_features=None, max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            presort=False, random_state=None, splitter='best')

In [162]:
big_model = tree.DecisionTreeClassifier(max_depth = 10)
big_model.fit(train_data.drop('safe_loans', axis = 1), train_data['safe_loans'])

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=10,
            max_features=None, max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            presort=False, random_state=None, splitter='best')

In [143]:
featurenames = loans1.drop('safe_loans', axis = 1)
features_names = featurenames.columns.values

In [144]:
with open("small_model.dot", "w") as f:
    f = tree.export_graphviz(small_model, out_file=f, 
                             feature_names= features_names 
                            )
    
import pydot

(graph,) = pydot.graph_from_dot_file('small_model.dot')
graph.write_png('small_model.png')

In [146]:
validation_safe_loans = validation_data[validation_data[target] == 1]
validation_risky_loans = validation_data[validation_data[target] == -1]

sample_validation_data_risky = validation_risky_loans[0:2]
sample_validation_data_safe = validation_safe_loans[0:2]

sample_validation_data = sample_validation_data_safe.append(sample_validation_data_risky)
sample_validation_data


,short_emp,emp_length_num,dti,last_delinq_none,last_major_derog_none,revol_util,total_rec_late_fee,safe_loans,grade_A,grade_B,...,purpose_house,purpose_major_purchase,purpose_medical,purpose_moving,purpose_other,purpose_small_business,purpose_vacation,purpose_wedding,term_ 36 months,term_ 60 months
19,0,11,11.18,1,1,82.4,0.0,1,0,1,...,0,0,0,0,0,0,0,0,1,0
79,0,10,16.85,1,1,96.4,0.0,1,0,0,...,0,0,0,0,0,0,0,0,1,0
24,0,3,13.97,0,1,59.5,0.0,-1,0,0,...,0,0,0,0,1,0,0,0,0,1
41,0,11,16.33,1,1,62.1,0.0,-1,1,0,...,0,0,0,0,0,0,0,0,1,0


In [125]:
decision_tree_model.predict(sample_validation_data.drop('safe_loans', axis = 1))

array([ 1, -1, -1,  1])

In [147]:
sample_validation_data['safe_loans']

19    1
79    1
24   -1
41   -1
Name: safe_loans, dtype: int64

In [148]:
decision_tree_model.predict_proba(sample_validation_data.drop('safe_loans', axis = 1))

array([[0.34156543, 0.65843457],
       [0.53630646, 0.46369354],
       [0.64750958, 0.35249042],
       [0.20789474, 0.79210526]])

In [154]:
temp = sample_validation_data.iloc[1,:]

In [156]:
small_model.predict_proba(sample_validation_data.drop('safe_loans', axis=1))

array([[0.41896585, 0.58103415],
       [0.59255339, 0.40744661],
       [0.59255339, 0.40744661],
       [0.23120112, 0.76879888]])

In [158]:
decision_tree_model.score(validation_data.drop('safe_loans', axis = 1), 
                          validation_data['safe_loans'])

0.6361482119775959

In [159]:
small_model.score(validation_data.drop('safe_loans', axis = 1), validation_data['safe_loans'])

0.6193451098664369

In [160]:
decision_tree_model.score(train_data.drop('safe_loans', axis = 1), train_data['safe_loans'])

0.6405276165914464

In [161]:
small_model.score(train_data.drop('safe_loans', axis = 1), 
                  train_data['safe_loans'])

0.6135020416935311

In [163]:
big_model.score(validation_data.drop('safe_loans', axis = 1), 
                          validation_data['safe_loans'])

0.6269926755708746

In [164]:
big_model.score(train_data.drop('safe_loans', axis = 1), 
                  train_data['safe_loans'])

0.6637384483129164

In [176]:
predictions = decision_tree_model.predict(
    validation_data.drop('safe_loans', axis = 1))

real = validation_data['safe_loans'].tolist()

In [178]:
total_cost = 0
for i in range(len(predictions)):
    if predictions[i] == -1 and real[i] == 1:
        total_cost = total_cost + 10000
    elif predictions[i] == 1 and real[i] == -1:
        total_cost = total_cost + 20000

In [179]:
total_cost

50390000

### Implement Binary Decision Tree

- Write a function to compute the number of misclassified examples in an intermediate node.
- Write a function to find the best feature to split on.
- Build a binary decision tree from scratch.
- Make predictions using the decision tree.
- Evaluate the accuracy of the decision tree.
- Visualize the decision at the root node.

Important Note: In this assignment, we will focus on building decision trees where the data contain only binary (0 or 1) features. This allows us to avoid dealing with:

- Multiple intermediate nodes in a split
- The thresholding issues of real-valued features.

In [3]:
loans = pd.read_csv('lending-club-data 2.csv', low_memory = False)

In [4]:
di = {1:-1, 0: 1}
loans['safe_loans'] = loans['bad_loans'].map(di)
loans = loans.drop('bad_loans', axis = 1)

In [5]:
features = ['grade',              # grade of the loan
            'term',               # the term of the loan
            'home_ownership',     # home_ownership status: own, mortgage or rent
            'emp_length',         # number of years of employment
           ]
target = 'safe_loans'

In [6]:
loans = loans[features+ [target]]

### One-hot encoding

In [7]:
features = loans.columns.values

categorical_variables = []
for feat_name, feat_type in zip(dict(loans.dtypes).keys(), dict(
    loans.dtypes).values()) :
    if feat_type == object:
        categorical_variables.append(feat_name)

for feature in categorical_variables:
    loans_data_one_hot_encoded = pd.get_dummies(loans[feature],
                                                prefix=feature)
    loans =pd.concat([loans,loans_data_one_hot_encoded],axis=1)
    del loans[feature] 


In [10]:
with open ('module-5-assignment-2-test-idx.json') as json_data:
    test_index = json.load(json_data)

with open ('module-5-assignment-2-train-idx.json') as json_data:
    train_index = json.load(json_data)
    
train_data = loans.iloc[train_index]
test_data = loans.iloc[test_index]

In [54]:
features = train_data.columns.values
features = features[features != target]
features

array(['grade_A', 'grade_B', 'grade_C', 'grade_D', 'grade_E', 'grade_F',
       'grade_G', 'term_ 36 months', 'term_ 60 months',
       'home_ownership_MORTGAGE', 'home_ownership_OTHER',
       'home_ownership_OWN', 'home_ownership_RENT', 'emp_length_1 year',
       'emp_length_10+ years', 'emp_length_2 years', 'emp_length_3 years',
       'emp_length_4 years', 'emp_length_5 years', 'emp_length_6 years',
       'emp_length_7 years', 'emp_length_8 years', 'emp_length_9 years',
       'emp_length_< 1 year'], dtype=object)

### Function to count number of mistakes while predicting majority class

In [33]:
def intermediate_node_num_mistakes(labels_in_node):
    # Corner case: If labels_in_node is empty, return 0
    if len(labels_in_node) == 0:
        return 0    
    # Count the number of 1's (safe loans)
    safe_num = np.count_nonzero(labels_in_node > 0)
    # Count the number of -1's (risky loans)
    bad_num = np.count_nonzero(labels_in_node < 0)
    # Return the number of mistakes that the majority classifier makes.
    return(min(safe_num, bad_num))

### Function to pick best feature to split on
The function best_splitting_feature takes 3 arguments:

- The data
- The features to consider for splits (a list of strings of column names to consider for splits)
- The name of the target/label column (string)

In [35]:
def best_splitting_feature(data, features, target):
    
    target_values = data[target]
    best_feature = None 
    best_error = 10   
    # Note: Since error is always <= 1, 
    # we should intialize it with something larger than 1.

    # Convert to float to make sure error gets computed correctly.
    num_data_points = float(len(data))  
    
    # Loop through each feature to consider splitting on that feature
    for feature in features:
        
        # The left split will have all data points where the feature value is 0
        left_split = data[data[feature] == 0]
        right_split =  data[data[feature] == 1]
            
        # Calculate the number of misclassified examples in the left split.
        left_mistakes = intermediate_node_num_mistakes(left_split[target])            
        right_mistakes = intermediate_node_num_mistakes(right_split[target]) 
            
        # Compute the classification error of this split.
        error = (left_mistakes+right_mistakes)/len(data)
        
        # If this is the best error we have found so far, store the feature as best_feature and the error as best_error
 
        if error < best_error:
            best_error = error
            best_feature = feature
        
    
    return best_feature # Return the best feature we found

### Build the Tree

Tree Node:
    { 
   - 'is_leaf'            : True/False.
   - 'prediction'         : Prediction at the leaf node.
   - 'left'               : (dictionary corresponding to the left tree).
   - 'right'              : (dictionary corresponding to the right tree).
   - 'splitting_feature'  : The feature that this node splits on
}


In [36]:
def create_leaf(target_values):    
    # Create a leaf node
    leaf = {'splitting_feature' : None,
            'left' : None,
            'right' : None,
            'is_leaf': True }  
   
    # Count the number of data points that are +1 and -1 in this node.
    num_ones = len(target_values[target_values == +1])
    num_minus_ones = len(target_values[target_values == -1])    

    # For the leaf node, set the prediction to be the majority class.
    # Store the predicted class (1 or -1) in leaf['prediction']
    if num_ones > num_minus_ones:
        leaf['prediction'] = 1      
    else:
        leaf['prediction'] = -1          

    return leaf 

We have provided a function that learns the decision tree recursively and implements 3 stopping conditions:

- Stopping condition 1: All data points in a node are from the same class.
- Stopping condition 2: No more features to split on.
- Additional stopping condition: max_depth of the tree

In [59]:
def decision_tree_create(data, features, target, current_depth = 0, max_depth = 10):
    remaining_features = features[:] # Make a copy of the features.
    
    target_values = data[target]
    print ("--------------------------------------------------------------------")
    print ("Subtree, depth = %s (%s data points)." % (current_depth, len(target_values)))
    

    # Stopping condition 1
    # All data points in a node are from the same class.

    mistake_num = intermediate_node_num_mistakes(target_values)
    if mistake_num == 0:  
        print ("Stopping condition 1 reached.")    
        # If not mistakes at current node, make current node a leaf node
        return create_leaf(target_values)
    
    # Stopping condition 2 
    # check if there are remaining features to consider splitting on
    if remaining_features == []:   
        print ("Stopping condition 2 reached.") 
        # If there are no remaining features to consider, make current node a leaf node
        return create_leaf(target_values)    
    
    # Additional stopping condition 
    # limit tree depth
    if current_depth >= max_depth:  
        print ("Reached maximum depth. Stopping for now.")
        # If the max tree depth has been reached, make current node a leaf node
        return create_leaf(target_values)

    # Find the best splitting feature (recall the function best_splitting_feature implemented above)
    splitting_feature = best_splitting_feature(data,remaining_features,target)
    # Split on the best feature that we found. 
    left_split = data[data[splitting_feature] == 0]
    right_split = data[data[splitting_feature] == 1]    
    remaining_features =  remaining_features[remaining_features!=splitting_feature]
    
    print ("Split on feature %s. (%s, %s)" % (\
                      splitting_feature, len(left_split), len(right_split)))
    
    # Create a leaf node if the split is "perfect"
    if len(left_split) == len(data):
        print ("Creating leaf node.")
        return create_leaf(left_split[target])
    
    if len(right_split) == len(data):
        print ("Creating leaf node.")
        return create_leaf(right_split[target])

        
    # Repeat (recurse) on left and right subtrees
    left_tree = decision_tree_create(left_split, remaining_features, target, current_depth + 1, max_depth)        
    ## YOUR CODE HERE
    right_tree = decision_tree_create(right_split, remaining_features, target, current_depth + 1, max_depth) 

    return {'is_leaf'          : False, 
            'prediction'       : None,
            'splitting_feature': splitting_feature,
            'left'             : left_tree, 
            'right'            : right_tree}

In [60]:
my_decision_tree = decision_tree_create(
                train_data, 
                features, 
                target, 
                current_depth = 0, 
                max_depth = 6)

--------------------------------------------------------------------
Subtree, depth = 0 (37224 data points).
Split on feature term_ 36 months. (9223, 28001)
--------------------------------------------------------------------
Subtree, depth = 1 (9223 data points).


/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:20: DeprecationWarning: elementwise == comparison failed; this will raise an error in the future.


Split on feature grade_A. (9122, 101)
--------------------------------------------------------------------
Subtree, depth = 2 (9122 data points).
Split on feature grade_B. (8074, 1048)
--------------------------------------------------------------------
Subtree, depth = 3 (8074 data points).
Split on feature grade_C. (5884, 2190)
--------------------------------------------------------------------
Subtree, depth = 4 (5884 data points).
Split on feature grade_D. (3826, 2058)
--------------------------------------------------------------------
Subtree, depth = 5 (3826 data points).
Split on feature grade_E. (1693, 2133)
--------------------------------------------------------------------
Subtree, depth = 6 (1693 data points).
Reached maximum depth. Stopping for now.
--------------------------------------------------------------------
Subtree, depth = 6 (2133 data points).
Reached maximum depth. Stopping for now.
--------------------------------------------------------------------
Subtree

In [82]:
def classify(tree, x, annotate = False):
       # if the node is a leaf node.
    if tree['is_leaf']:
        if annotate:
             print ("At leaf, predicting %s" % tree['prediction'])
        return tree['prediction']
    else:
        # split on feature.
        split_feature_value = x[tree['splitting_feature']]
        if annotate:
             print ("Split on %s = %s" % (tree['splitting_feature'], split_feature_value))
        if split_feature_value == 0:
            return classify(tree['left'], x, annotate)
        else:
            return classify(tree['right'], x, annotate)

In [64]:
print (test_data.iloc[0,])
print ('Predicted class: %s ' % classify(my_decision_tree, test_data.iloc[0,]))

safe_loans                -1
grade_A                    0
grade_B                    0
grade_C                    0
grade_D                    1
grade_E                    0
grade_F                    0
grade_G                    0
term_ 36 months            0
term_ 60 months            1
home_ownership_MORTGAGE    0
home_ownership_OTHER       0
home_ownership_OWN         0
home_ownership_RENT        1
emp_length_1 year          0
emp_length_10+ years       0
emp_length_2 years         1
emp_length_3 years         0
emp_length_4 years         0
emp_length_5 years         0
emp_length_6 years         0
emp_length_7 years         0
emp_length_8 years         0
emp_length_9 years         0
emp_length_< 1 year        0
Name: 24, dtype: int64
Predicted class: -1 


In [65]:
classify(my_decision_tree, test_data.iloc[0,], annotate=True)

Split on term_ 36 months = 0
Split on grade_A = 0
Split on grade_B = 0
Split on grade_C = 0
Split on grade_D = 1
At leaf, predicting -1


-1

### Evaluating your decision tree

- tree (as described above)
- data (a data frame of data points)

In [81]:
def evaluate_classification_error(tree, data):
    # Apply the classify(tree, x) to each row in your data
    prediction = data.apply(lambda x: classify(tree, x), axis = 1)
    result = data[target]
    
    compare = prediction == result
    num_correct = np.count_nonzero(compare)
    error_rate = 1-num_correct/len(result)
    return error_rate

In [73]:
test_data = test_data.fillna(0)

In [83]:
evaluate_classification_error(my_decision_tree, test_data)

0.37774666092201636

### Print the Decision Stump

In [87]:
def print_stump(tree, name = 'root'):
    split_name = tree['splitting_feature'] 
  
    if split_name is None:
        print ("(leaf, label: %s)" % tree['prediction'])
        return None
    #split_feature, split_value = split_name.split('.')
    
    print ('                       %s' % name         )
    print ('         |---------------|----------------|')
    print ('         |                                |')
    print ('         |                                |')
    print ('         |                                |')
    print ('  [{0} == 0]               [{0} == 1]    '.format(split_name))
    print ('         |                                |')
    print ('         |                                |')
    print ('         |                                |')
    print ('    (%s)                         (%s)' \
        % (('leaf, label: ' + str(tree['left']['prediction']) if 
            tree['left']['is_leaf'] else 'subtree'),
           ('leaf, label: ' + str(tree['right']['prediction']) if 
            tree['right']['is_leaf'] else 'subtree')))

In [88]:
print_stump(my_decision_tree)

                       root
         |---------------|----------------|
         |                                |
         |                                |
         |                                |
  [term_ 36 months == 0]               [term_ 36 months == 1]    
         |                                |
         |                                |
         |                                |
    (subtree)                         (subtree)


In [89]:
print_stump(my_decision_tree['left'], 
            my_decision_tree['splitting_feature'])

                       term_ 36 months
         |---------------|----------------|
         |                                |
         |                                |
         |                                |
  [grade_A == 0]               [grade_A == 1]    
         |                                |
         |                                |
         |                                |
    (subtree)                         (subtree)


In [90]:
print_stump(my_decision_tree['left']['left'], 
            my_decision_tree['left']['splitting_feature'])

                       grade_A
         |---------------|----------------|
         |                                |
         |                                |
         |                                |
  [grade_B == 0]               [grade_B == 1]    
         |                                |
         |                                |
         |                                |
    (subtree)                         (subtree)


In [91]:
print_stump(my_decision_tree['right'], 
            my_decision_tree['splitting_feature'])

                       term_ 36 months
         |---------------|----------------|
         |                                |
         |                                |
         |                                |
  [grade_D == 0]               [grade_D == 1]    
         |                                |
         |                                |
         |                                |
    (subtree)                         (leaf, label: -1)


In [92]:
print_stump(my_decision_tree['right']['right'], 
            my_decision_tree['right']['splitting_feature'])

(leaf, label: -1)


### Decision Tree in Practice

- Implement binary decision trees with different early stopping methods.
- Compare models with different stopping parameters.
- Visualize the concept of overfitting in decision trees.

In [95]:
loans3 = pd.read_csv('lending-club-data 3.csv', low_memory = False)

In [97]:
loans3['safe_loans'] = loans3['bad_loans'].apply(lambda x : +1 if x==0 else -1)
loans3 = loans3.drop('bad_loans', axis = 1)

In [99]:
features = ['grade',              # grade of the loan
            'term',               # the term of the loan
            'home_ownership',     # home_ownership status: own, mortgage or rent
            'emp_length',         # number of years of employment
           ]
target = 'safe_loans'

loans3 = loans3[features+[target]]

In [101]:
# Extract categorical columns
categorical_variables = []
for feat_name, feat_type in zip(dict(loans3.dtypes).keys(), dict(
    loans3.dtypes).values()) :
    if feat_type == object:
        categorical_variables.append(feat_name)

for feature in categorical_variables:
    loans_data_one_hot_encoded = pd.get_dummies(loans3[feature],
                                                prefix=feature)
    loans3 =pd.concat([loans3,loans_data_one_hot_encoded],axis=1)
    del loans3[feature] 
   

In [103]:
loans3.head(2)

,safe_loans,grade_A,grade_B,grade_C,grade_D,grade_E,grade_F,grade_G,term_ 36 months,term_ 60 months,...,emp_length_10+ years,emp_length_2 years,emp_length_3 years,emp_length_4 years,emp_length_5 years,emp_length_6 years,emp_length_7 years,emp_length_8 years,emp_length_9 years,emp_length_< 1 year
0,1,0,1,0,0,0,0,0,1,0,...,1,0,0,0,0,0,0,0,0,0
1,-1,0,0,1,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,1


In [105]:
with open ('module-6-assignment-train-idx.json') as json_data:
    train_idx = json.load(json_data)
    
with open ('module-6-assignment-validation-idx.json') as json_data:
    validation_idx = json.load(json_data)   
    
train_data = loans3.iloc[train_idx]
validation_data = loans3.iloc[validation_idx]

### Early stopping methods for decision trees

- Reached a maximum depth. (set by parameter max_depth).
- Reached a minimum node size. (set by parameter min_node_size).
- Don't split if the gain in error reduction is too small. (set by parameter min_error_reduction).

#### Early stopping condition 1: Maximum depth

#### Early stopping condition 2: Minimum node size


The function reached_minimum_node_size takes 2 arguments:

- The data (from a node)
- The minimum number of data points that a node is allowed to split on, min_node_size.


In [106]:
def reached_minimum_node_size(data, min_node_size):
    # Return True if the number of data points <= minimum node size.
    return len(data)<= min_node_size

#### Early stopping condition 3: Minimum gain in error reduction
The function error_reduction takes 2 arguments:

- The error before a split, error_before_split.
- The error after a split, error_after_split.


In [107]:
def error_reduction(error_before_split, error_after_split):
    # Return the error before the split minus the error after the split.
    return error_before_split - error_after_split

In [118]:
def decision_tree_create(data, features, target, current_depth = 0, 
                         max_depth = 10, min_node_size=1, 
                         min_error_reduction=0.0):
    
    remaining_features = features[:] # Make a copy of the features.
    
    target_values = data[target]
    print ("--------------------------------------------------------------------")
    print ("Subtree, depth = %s (%s data points)." % (current_depth, len(target_values)))
    
    
    # Stopping condition 1: All nodes are of the same type.
    if intermediate_node_num_mistakes(target_values) == 0:
        print ("Stopping condition 1 reached. All data points have the same target value.")                
        return create_leaf(target_values)
    
    # Stopping condition 2: No more features to split on.
    if remaining_features == []:
        print ("Stopping condition 2 reached. No remaining features." )               
        return create_leaf(target_values)    
    
    # Early stopping condition 1: Reached max depth limit.
    if current_depth >= max_depth:
        print ("Early stopping condition 1 reached. Reached maximum depth.")
        return create_leaf(target_values)
    
    # Early stopping condition 2: Reached the minimum node size.
    # If the number of data points is less than or equal to the minimum size, return a leaf.
    if reached_minimum_node_size(data,min_node_size):          
        print ("Early stopping condition 2 reached. Reached minimum node size.")
        return create_leaf(target_values)
    
    # Find the best splitting feature
    splitting_feature = best_splitting_feature(data, features, target)
    
    # Split on the best feature that we found. 
    left_split = data[data[splitting_feature] == 0]
    right_split = data[data[splitting_feature] == 1]
    
    if len(left_split) == 0 or len(right_split) ==0:
        return create_leaf(target_values)
    # Early stopping condition 3: Minimum error reduction
    # Calculate the error before splitting (number of misclassified examples 
    # divided by the total number of examples)
    error_before_split = intermediate_node_num_mistakes(target_values) / float(len(data))
    
    # Calculate the error after splitting (number of misclassified examples 
    # in both groups divided by the total number of examples)
    left_mistakes = intermediate_node_num_mistakes(left_split[target])/ float(len(left_split))
    right_mistakes = intermediate_node_num_mistakes(right_split[target])/float(len(right_split))
    error_after_split = (left_mistakes + right_mistakes) / float(len(data))
    
    # If the error reduction is <=  min_error_reduction, return a leaf.
    if error_reduction(error_before_split, error_after_split) <= min_error_reduction:
        print ("Early stopping condition 3 reached. Minimum error reduction.")
        return  create_leaf(target_values)
    
    remaining_features = remaining_features[remaining_features!=splitting_feature]
    print ("Split on feature %s. (%s, %s)" % (\
                      splitting_feature, len(left_split), len(right_split)))
    
    
    # Repeat (recurse) on left and right subtrees
    left_tree = decision_tree_create(left_split, remaining_features, target, 
                                     current_depth + 1, max_depth, 
                                     min_node_size, min_error_reduction)        
    
    ## YOUR CODE HERE
    right_tree = decision_tree_create(right_split, remaining_features, target, 
                                     current_depth + 1, max_depth, 
                                     min_node_size, min_error_reduction) 
    
    
    return {'is_leaf'          : False, 
            'prediction'       : None,
            'splitting_feature': splitting_feature,
            'left'             : left_tree, 
            'right'            : right_tree}

In [116]:
features = test_data.columns.values
features = features[features!='safe_loans']
features

array(['grade_A', 'grade_B', 'grade_C', 'grade_D', 'grade_E', 'grade_F',
       'grade_G', 'term_ 36 months', 'term_ 60 months',
       'home_ownership_MORTGAGE', 'home_ownership_OTHER',
       'home_ownership_OWN', 'home_ownership_RENT', 'emp_length_1 year',
       'emp_length_10+ years', 'emp_length_2 years', 'emp_length_3 years',
       'emp_length_4 years', 'emp_length_5 years', 'emp_length_6 years',
       'emp_length_7 years', 'emp_length_8 years', 'emp_length_9 years',
       'emp_length_< 1 year'], dtype=object)

In [119]:
my_decision_tree_new = decision_tree_create(train_data, features, 'safe_loans', max_depth = 6, 
                                min_node_size = 100, min_error_reduction=0.0)

--------------------------------------------------------------------
Subtree, depth = 0 (37224 data points).
Split on feature term_ 36 months. (9223, 28001)
--------------------------------------------------------------------
Subtree, depth = 1 (9223 data points).


/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:18: DeprecationWarning: elementwise == comparison failed; this will raise an error in the future.


Split on feature grade_A. (9122, 101)
--------------------------------------------------------------------
Subtree, depth = 2 (9122 data points).
Split on feature grade_B. (8074, 1048)
--------------------------------------------------------------------
Subtree, depth = 3 (8074 data points).
Split on feature grade_C. (5884, 2190)
--------------------------------------------------------------------
Subtree, depth = 4 (5884 data points).
Split on feature grade_D. (3826, 2058)
--------------------------------------------------------------------
Subtree, depth = 5 (3826 data points).
Split on feature grade_E. (1693, 2133)
--------------------------------------------------------------------
Subtree, depth = 6 (1693 data points).
Early stopping condition 1 reached. Reached maximum depth.
--------------------------------------------------------------------
Subtree, depth = 6 (2133 data points).
Early stopping condition 1 reached. Reached maximum depth.
----------------------------------------

In [121]:
print (validation_data.iloc[0,])
print ('Predicted class: %s ' % classify(my_decision_tree_new, validation_data.iloc[0,]))

safe_loans                -1
grade_A                    0
grade_B                    0
grade_C                    0
grade_D                    1
grade_E                    0
grade_F                    0
grade_G                    0
term_ 36 months            0
term_ 60 months            1
home_ownership_MORTGAGE    0
home_ownership_OTHER       0
home_ownership_OWN         0
home_ownership_RENT        1
emp_length_1 year          0
emp_length_10+ years       0
emp_length_2 years         1
emp_length_3 years         0
emp_length_4 years         0
emp_length_5 years         0
emp_length_6 years         0
emp_length_7 years         0
emp_length_8 years         0
emp_length_9 years         0
emp_length_< 1 year        0
Name: 24, dtype: int64
Predicted class: -1 


In [122]:
my_decision_tree_old = decision_tree_create(train_data, features, 'safe_loans', max_depth = 6,                                 
                                            min_node_size = 0, min_error_reduction=-1)

--------------------------------------------------------------------
Subtree, depth = 0 (37224 data points).
Split on feature term_ 36 months. (9223, 28001)
--------------------------------------------------------------------
Subtree, depth = 1 (9223 data points).


/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:18: DeprecationWarning: elementwise == comparison failed; this will raise an error in the future.


Split on feature grade_A. (9122, 101)
--------------------------------------------------------------------
Subtree, depth = 2 (9122 data points).
Split on feature grade_B. (8074, 1048)
--------------------------------------------------------------------
Subtree, depth = 3 (8074 data points).
Split on feature grade_C. (5884, 2190)
--------------------------------------------------------------------
Subtree, depth = 4 (5884 data points).
Split on feature grade_D. (3826, 2058)
--------------------------------------------------------------------
Subtree, depth = 5 (3826 data points).
Split on feature grade_E. (1693, 2133)
--------------------------------------------------------------------
Subtree, depth = 6 (1693 data points).
Early stopping condition 1 reached. Reached maximum depth.
--------------------------------------------------------------------
Subtree, depth = 6 (2133 data points).
Early stopping condition 1 reached. Reached maximum depth.
----------------------------------------

In [124]:
classify(my_decision_tree_old, validation_data.iloc[0,], annotate = True)

Split on term_ 36 months = 0
Split on grade_A = 0
Split on grade_B = 0
Split on grade_C = 0
Split on grade_D = 1
At leaf, predicting -1


-1

In [129]:
evaluate_classification_error(my_decision_tree_new, validation_data)

0.37817750969409736

In [130]:
evaluate_classification_error(my_decision_tree_old, validation_data)

0.37774666092201636

### Exploring the effect of max_depth

 Train three models with these parameters:

- model_1: max_depth = 2 (too small)
- model_2: max_depth = 6 (just right)
- model_3: max_depth = 14 (may be too large)

In [132]:
 model_1 = decision_tree_create(train_data, features, 'safe_loans', max_depth = 2, 
                                min_node_size = 0, min_error_reduction=-1)

--------------------------------------------------------------------
Subtree, depth = 0 (37224 data points).
Split on feature term_ 36 months. (9223, 28001)
--------------------------------------------------------------------
Subtree, depth = 1 (9223 data points).


/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:18: DeprecationWarning: elementwise == comparison failed; this will raise an error in the future.


Split on feature grade_A. (9122, 101)
--------------------------------------------------------------------
Subtree, depth = 2 (9122 data points).
Early stopping condition 1 reached. Reached maximum depth.
--------------------------------------------------------------------
Subtree, depth = 2 (101 data points).
Early stopping condition 1 reached. Reached maximum depth.
--------------------------------------------------------------------
Subtree, depth = 1 (28001 data points).
Split on feature grade_D. (23300, 4701)
--------------------------------------------------------------------
Subtree, depth = 2 (23300 data points).
Early stopping condition 1 reached. Reached maximum depth.
--------------------------------------------------------------------
Subtree, depth = 2 (4701 data points).
Early stopping condition 1 reached. Reached maximum depth.


In [133]:
 model_2 = decision_tree_create(train_data, features, 'safe_loans', 
                                max_depth = 6, 
                                min_node_size = 0, min_error_reduction=-1)

--------------------------------------------------------------------
Subtree, depth = 0 (37224 data points).
Split on feature term_ 36 months. (9223, 28001)
--------------------------------------------------------------------
Subtree, depth = 1 (9223 data points).


/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:18: DeprecationWarning: elementwise == comparison failed; this will raise an error in the future.


Split on feature grade_A. (9122, 101)
--------------------------------------------------------------------
Subtree, depth = 2 (9122 data points).
Split on feature grade_B. (8074, 1048)
--------------------------------------------------------------------
Subtree, depth = 3 (8074 data points).
Split on feature grade_C. (5884, 2190)
--------------------------------------------------------------------
Subtree, depth = 4 (5884 data points).
Split on feature grade_D. (3826, 2058)
--------------------------------------------------------------------
Subtree, depth = 5 (3826 data points).
Split on feature grade_E. (1693, 2133)
--------------------------------------------------------------------
Subtree, depth = 6 (1693 data points).
Early stopping condition 1 reached. Reached maximum depth.
--------------------------------------------------------------------
Subtree, depth = 6 (2133 data points).
Early stopping condition 1 reached. Reached maximum depth.
----------------------------------------

In [134]:
 model_3 = decision_tree_create(train_data, features, 'safe_loans', 
                                max_depth = 14, 
                                min_node_size = 0, min_error_reduction=-1)

--------------------------------------------------------------------
Subtree, depth = 0 (37224 data points).
Split on feature term_ 36 months. (9223, 28001)
--------------------------------------------------------------------
Subtree, depth = 1 (9223 data points).


/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:18: DeprecationWarning: elementwise == comparison failed; this will raise an error in the future.


Split on feature grade_A. (9122, 101)
--------------------------------------------------------------------
Subtree, depth = 2 (9122 data points).
Split on feature grade_B. (8074, 1048)
--------------------------------------------------------------------
Subtree, depth = 3 (8074 data points).
Split on feature grade_C. (5884, 2190)
--------------------------------------------------------------------
Subtree, depth = 4 (5884 data points).
Split on feature grade_D. (3826, 2058)
--------------------------------------------------------------------
Subtree, depth = 5 (3826 data points).
Split on feature grade_E. (1693, 2133)
--------------------------------------------------------------------
Subtree, depth = 6 (1693 data points).
Split on feature home_ownership_OTHER. (1692, 1)
--------------------------------------------------------------------
Subtree, depth = 7 (1692 data points).
Split on feature grade_F. (339, 1353)
--------------------------------------------------------------------
Su

In [135]:
print(evaluate_classification_error(model_1, train_data))
print(evaluate_classification_error(model_2, train_data))
print(evaluate_classification_error(model_3, train_data))

0.4000376101439931
0.38042660649043625
0.37892220073071137


In [136]:
print(evaluate_classification_error(model_1, validation_data))
print(evaluate_classification_error(model_2, validation_data))
print(evaluate_classification_error(model_3, validation_data))

0.3981042654028436
0.37774666092201636
0.37882378285221885


### Measuring the complexity of the tree

In [137]:
# Count leave numbers

def count_leaves(tree):
    if tree['is_leaf']:
        return 1
    return count_leaves(tree['left']) + count_leaves(tree['right'])

In [138]:
print(count_leaves(model_1))
print(count_leaves(model_2))
print(count_leaves(model_3))

4
18
35


### Exploring the effect of min_error

compare three models trained with different values of the stopping criterion

- model_4: min_error_reduction = -1 (ignoring this early stopping condition)
- model_5: min_error_reduction = 0 (just right)
- model_6: min_error_reduction = 5 (too positive)

In [147]:
 model_4 = decision_tree_create(train_data, features, 'safe_loans', 
                                max_depth = 6, 
                                min_node_size = 0, 
                                min_error_reduction=-1)

--------------------------------------------------------------------
Subtree, depth = 0 (37224 data points).
Split on feature term_ 36 months. (9223, 28001)
--------------------------------------------------------------------
Subtree, depth = 1 (9223 data points).


/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:18: DeprecationWarning: elementwise == comparison failed; this will raise an error in the future.


Split on feature grade_A. (9122, 101)
--------------------------------------------------------------------
Subtree, depth = 2 (9122 data points).
Split on feature grade_B. (8074, 1048)
--------------------------------------------------------------------
Subtree, depth = 3 (8074 data points).
Split on feature grade_C. (5884, 2190)
--------------------------------------------------------------------
Subtree, depth = 4 (5884 data points).
Split on feature grade_D. (3826, 2058)
--------------------------------------------------------------------
Subtree, depth = 5 (3826 data points).
Split on feature grade_E. (1693, 2133)
--------------------------------------------------------------------
Subtree, depth = 6 (1693 data points).
Early stopping condition 1 reached. Reached maximum depth.
--------------------------------------------------------------------
Subtree, depth = 6 (2133 data points).
Early stopping condition 1 reached. Reached maximum depth.
----------------------------------------

In [148]:
 model_5 = decision_tree_create(train_data, features, 'safe_loans', 
                                max_depth = 6, 
                                min_node_size = 0, 
                                min_error_reduction=0.0)

--------------------------------------------------------------------
Subtree, depth = 0 (37224 data points).
Split on feature term_ 36 months. (9223, 28001)
--------------------------------------------------------------------
Subtree, depth = 1 (9223 data points).


/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:18: DeprecationWarning: elementwise == comparison failed; this will raise an error in the future.


Split on feature grade_A. (9122, 101)
--------------------------------------------------------------------
Subtree, depth = 2 (9122 data points).
Split on feature grade_B. (8074, 1048)
--------------------------------------------------------------------
Subtree, depth = 3 (8074 data points).
Split on feature grade_C. (5884, 2190)
--------------------------------------------------------------------
Subtree, depth = 4 (5884 data points).
Split on feature grade_D. (3826, 2058)
--------------------------------------------------------------------
Subtree, depth = 5 (3826 data points).
Split on feature grade_E. (1693, 2133)
--------------------------------------------------------------------
Subtree, depth = 6 (1693 data points).
Early stopping condition 1 reached. Reached maximum depth.
--------------------------------------------------------------------
Subtree, depth = 6 (2133 data points).
Early stopping condition 1 reached. Reached maximum depth.
----------------------------------------

In [149]:
 model_6 = decision_tree_create(train_data, features, 'safe_loans', 
                                max_depth = 6, 
                                min_node_size = 0, 
                                min_error_reduction=5)

--------------------------------------------------------------------
Subtree, depth = 0 (37224 data points).
Early stopping condition 3 reached. Minimum error reduction.


/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:18: DeprecationWarning: elementwise == comparison failed; this will raise an error in the future.


In [150]:
print(evaluate_classification_error(model_4, validation_data))
print(evaluate_classification_error(model_5, validation_data))
print(evaluate_classification_error(model_6, validation_data))

0.37774666092201636
0.37774666092201636
0.503446790176648


In [151]:
print(count_leaves(model_4))
print(count_leaves(model_5))
print(count_leaves(model_6))

18
18
1


### Exploring the effect of min_node_size


compare three models trained with different values of the stopping criterion

- model_7: min_node_size = 0 (too small)
- model_8: min_node_size = 2000 (just right)
- model_9: min_node_size = 50000 (too large)

In [152]:
model_7 = decision_tree_create(train_data, features, 'safe_loans', 
                                max_depth = 6, 
                                min_node_size = 0, 
                                min_error_reduction=-1)
model_8 = decision_tree_create(train_data, features, 'safe_loans', 
                                max_depth = 6, 
                                min_node_size = 2000, 
                                min_error_reduction=-1)
model_9 = decision_tree_create(train_data, features, 'safe_loans', 
                                max_depth = 6, 
                                min_node_size = 50000, 
                                min_error_reduction=-1)

--------------------------------------------------------------------
Subtree, depth = 0 (37224 data points).
Split on feature term_ 36 months. (9223, 28001)
--------------------------------------------------------------------
Subtree, depth = 1 (9223 data points).


/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:18: DeprecationWarning: elementwise == comparison failed; this will raise an error in the future.


Split on feature grade_A. (9122, 101)
--------------------------------------------------------------------
Subtree, depth = 2 (9122 data points).
Split on feature grade_B. (8074, 1048)
--------------------------------------------------------------------
Subtree, depth = 3 (8074 data points).
Split on feature grade_C. (5884, 2190)
--------------------------------------------------------------------
Subtree, depth = 4 (5884 data points).
Split on feature grade_D. (3826, 2058)
--------------------------------------------------------------------
Subtree, depth = 5 (3826 data points).
Split on feature grade_E. (1693, 2133)
--------------------------------------------------------------------
Subtree, depth = 6 (1693 data points).
Early stopping condition 1 reached. Reached maximum depth.
--------------------------------------------------------------------
Subtree, depth = 6 (2133 data points).
Early stopping condition 1 reached. Reached maximum depth.
----------------------------------------

In [153]:
print(evaluate_classification_error(model_7, validation_data))
print(evaluate_classification_error(model_8, validation_data))
print(evaluate_classification_error(model_9, validation_data))

0.37774666092201636
0.3774235243429557
0.503446790176648


In [154]:
print(count_leaves(model_7))
print(count_leaves(model_8))
print(count_leaves(model_9))

18
13
1
